### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

# A notebook to test running RBAs at the voxel level on fMRI data.
Brainhack-Aus 2022 project

Authors:
Gang Chen @afni-gangc
Christopher Nolan @crnolan
Kelly Garner @kel-github 
Lea Waller @HippocampusGirl
Daniel Tomasz @danieltomasz
Megan Campbell @meganEJcampbell
Preetom Pal @preqon
Adam @a-manoogian *
Bella @isabellaorlando *
Darin Leiter @dsleiter *
Arshiyan @Arshiyasan *
Judy Zhu @jd-zhu 


Modelling task-based fMRI data often involves performing a GLM at each voxel and then correcting for many many many multiple comparisons.

Here instead, we try performing a single hierarchical mixed effects model on all the voxels at once.

This provides advantages typical of Bayesian hiearchal modelling; information at upper levels of the hierarchy (e.g. across voxels) can help inform estimates at lower levels (the estimate for each voxel) - aka shrinkage - and we avoid the multiple comparisons problem by instead providing the strength of evidenve for the effect of interest at each voxel.

For a comprehensive introduction to this approach, see this paper and this paper by Gang Chen.

Here we test the feasibility of running Bayesian hierarchal modelling at the voxel level, by determining compute time across varying data sizes; both randomly generated and fMRI data.

# Running the notebook

To build the environment to run this notebook, follow the instructions [here](https://github.com/crnolan/pyrba)

# Import modules

In [1]:
import arviz as az
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import numpy as np
import pandas as pd
import pymc as pm
import xarray as xr
import bambi as bmb
sns.set_theme(style='white')

# Generate data

Here is an example of how to generate a test dataset. We are simulating voxel data from a task with 2 conditions (e.g. go and stop conditions of a stop signal task), with additive values per voxel and per subject. 

In [2]:
# define the number of voxels and subjects for which you wish to simulate data
nvoxels = 1000
nsubs = 30
mean_cond_a = .5
mean_cond_b = .3
sd_cond_a = .15
sd_cond_b = .15

In [3]:
#create voxel noise parameters
noise_list = []
for i in range(0,nvoxels): #hardcoded
    mean =  np.random.normal(0, .08) #voxel noise paramerer - normal distribution
    sd =  abs(np.random.normal(0, .05)) #voxel noise paramerer - uniform distribution or absolute normal distribution
    noise_list.append([mean, sd])


In [4]:
noise_list

[[0.04913122353418864, 0.08891056439666284],
 [0.02760758973597702, 0.028282926739449217],
 [-0.08533328580730447, 0.020267326537369998],
 [-0.11686644684898498, 0.11587008914274695],
 [-0.007462143550040829, 0.05275681522293753],
 [0.0343458699692995, 0.05604779149702075],
 [0.056825613498936514, 0.10929100012262086],
 [0.07813243297143913, 0.017700704463077543],
 [-0.07666610510840575, 0.014732069536706392],
 [-0.05256198837542331, 0.011469350277493937],
 [0.03165500050113462, 0.007695664178766666],
 [0.1489002327335648, 0.12777067623687507],
 [0.010072646641741994, 0.04028144742297193],
 [-0.0926775459324589, 0.05162051425818823],
 [-0.11501196020419029, 0.05046170948953216],
 [0.06844593595195257, 0.06379545268115212],
 [-0.04482940283134587, 0.007605244742954065],
 [0.030994889467743803, 0.007121323384758543],
 [-0.07933130649858702, 0.022532058179866383],
 [0.02410889045249195, 0.056379151319009696],
 [-0.012185640123421188, 0.061803493863839126],
 [0.14262257723246355, 0.0366881

In [5]:
#define function to generate random voxel values
def generate_random_voxels(mean, sd, noise_list, length=nvoxels): #hardcoded
    # mean [1 value] - reflects the mean of the condition + subject
    # sd [as above, but sd]
    # noise_list arr[nvoxels, 2] rfx for each voxel
    voxels = []
    for v in range(length):
        mean = mean + noise_list[v][0] + np.random.normal(0, 0.2)# mean = condition + subrfx + voxel mean + residual noise
        sd = sd + noise_list[v][1] # as above but with sd
        voxels.append(np.random.normal(mean,sd)) 
    return voxels

In [6]:
participants = []
conditions = []

In [7]:
#create multi level index matrix
for i in range(nsubs):
    participants.append(i)
    participants.append(i)
    conditions.append(0)
    conditions.append(1)
arrays = [participants, conditions]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=["participant", "condition"])

In [8]:
#initiate voxel list
data = np.zeros((nsubs*2,nvoxels)) #hardcoded to be *2 participants (for 2 conditions)
df = pd.DataFrame(data, index = multi_index)

In [ ]:
data.shape

In [9]:
#populate the multi level index matrix
for participant in range(nsubs):
    # unique number for each paritipcant
    random_effect_mean = np.random.normal(0, .1) # drawn from common distribution
    random_effect_sd = abs(np.random.normal(0, .05))
    
    for condition in range(2): # 2 conditions
        if condition == 0:
            mean = mean_cond_a + random_effect_mean
            sd = sd_cond_a + random_effect_sd
            df.loc[participant, condition] = generate_random_voxels(mean,sd,noise_list) 
        if condition == 1:
            mean = mean_cond_b + random_effect_mean
            sd = sd_cond_b + random_effect_sd
            df.loc[participant, condition] = generate_random_voxels(mean,sd,noise_list) 

In [ ]:
df

In [10]:
#melt to satisfy bambi long form
df = pd.melt(df, ignore_index=False, var_name="voxel_id", value_name = "BOLD")

df


voxel_id       BOLD
participant condition                     
0           0                 0   0.704385
            1                 0   0.345577
1           0                 0   0.194460
            1                 0   0.218984
2           0                 0  -0.058261
...                         ...        ...
27          1               999 -59.845411
28          0               999 -50.076503
            1               999  22.136042
29          0               999 -10.455338
            1               999 -36.397224

[60000 rows x 2 columns]

In [11]:
#write to csv
tmp = df


In [13]:
tmp.head(5)

voxel_id      BOLD
participant condition                    
0           0                 0  0.704385
            1                 0  0.345577
1           0                 0  0.194460
            1                 0  0.218984
2           0                 0 -0.058261

In [15]:
tmp.to_csv('df2.txt', sep=' ')

In [16]:
df = pd.read_csv('df2.txt', delimiter = ' ')
df.head(5)

,participant,condition,voxel_id,BOLD
0,0,0,0,0.704385
1,0,1,0,0.345577
2,1,0,0,0.194460
3,1,1,0,0.218984
4,2,0,0,-0.058261


Now I want to define the following model to apply to the data:


In [17]:
model = bmb.Model("BOLD ~ condition + (1|participant) + (1|voxel_id)", data=df)

In [ ]:
%%time

fitted = model.fit(tune=4000, 
                   draws=1000, 
                   chains=16, 
                   method='nuts_numpyro',
                   nuts_kwargs=dict(max_tree_depth=100))

/home/datascience/.conda/envs/pyrba/lib/python3.9/site-packages/bambi/models.py:260: FutureWarning: the method argument has been deprecated, please use inference_method
  warnings.warn(
/home/datascience/.conda/envs/pyrba/lib/python3.9/site-packages/pymc/sampling_jax.py:37: UserWarning: This module is experimental.
  warnings.warn("This module is experimental.")


Compiling...


Compilation time =  0:01:34.194503
Sampling...


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

...

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
model.graph()

In [ ]:
az.plot_trace(fitted, figsize=(20, 35))
az.summary(fitted)